# 🧠 Model Training

This notebook is designed to be deployed into a containerized node.

## 1. Python Setup

Run in the terminal:
```
python -m venv ultralytics-env
. ultralytics-env/bin/activate
pip install -r dock_requirements.txt
```

In [1]:
from ultralytics import YOLO
import gdown
from dotenv import load_dotenv
import os
from zipfile import ZipFile
import pandas as pd
import numpy as np
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/developer/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 2. Download Data Files

In [2]:
# Create a folder to download zip files to
os.makedirs(os.path.join('data', 'downloads'), exist_ok=True)
# Create the processed data folder
os.makedirs(os.path.join('data', 'processed'), exist_ok=True)

In [3]:
# Load environment variables from .env file
load_dotenv('secrets.env')

True

In [4]:
# Set download variables
BASELINE_DATA_ID = os.getenv('BASELINE')
MODEL_3D_DATA_ID = os.getenv('3D_MODELS')
CLIPART_DATA_ID = os.getenv('CLIPART')
GENAI_DATA_ID = os.getenv('GENAI')
TEST_DATA_ID = os.getenv('TEST_DATA')

In [5]:
# Downloads base URL
BASE_URL = "https://drive.google.com/file/d/"
URL_SUFFIX = "/view?usp=sharing"
# Download path
DOWNLOAD_PATH = os.path.join('data', 'downloads')

If downloads aren't working it may be necessary to download your cookies for google drive and placing it in the `~/.cache/gdown/` folder.

```
Place cookies file in working directory
Use: mv cookies.txt ~/.cache/gdown/cookies.txt
Retry download
```

In [6]:
# Download the test data set
gdown.download(f"{BASE_URL}{TEST_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, 'test_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1YrQL8xF59OqhaM-xssrwEOTJi8JIM759
From (redirected): https://drive.google.com/uc?id=1YrQL8xF59OqhaM-xssrwEOTJi8JIM759&confirm=t&uuid=350b40ee-21fe-4bc0-b6cd-f9bbfc66ce13&at=ABOpFIrSe_PrXfvCCuMNNj77LMEj%3A1761837148433
To: /home/developer/synth-data-proj/data/downloads/test_data.zip
100%|██████████| 2.43G/2.43G [01:13<00:00, 33.2MB/s]


'data/downloads/test_data.zip'

In [7]:
# Download the baseline data set
gdown.download(f"{BASE_URL}{BASELINE_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, 'baseline_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1-3U-WfRgL3krWNlIoZOJ4qipGywepOxg
From (redirected): https://drive.google.com/uc?id=1-3U-WfRgL3krWNlIoZOJ4qipGywepOxg&confirm=t&uuid=0770de7a-c629-4637-9c27-91a3168ffd3c&at=ABOpFIpyKJE1Pha6mloUgCLvsTey%3A1761837269577
To: /home/developer/synth-data-proj/data/downloads/baseline_data.zip
100%|██████████| 26.1G/26.1G [17:18<00:00, 25.1MB/s]  


'data/downloads/baseline_data.zip'

In [8]:
# Download the 3D model data set
gdown.download(f"{BASE_URL}{MODEL_3D_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, '3D_model_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1kHtr0K5ImHuZwEi_xktJQOGbD4x2PdEg
From (redirected): https://drive.google.com/uc?id=1kHtr0K5ImHuZwEi_xktJQOGbD4x2PdEg&confirm=t&uuid=d67d635d-0e59-4d8b-8c74-8b175be20748
To: /home/developer/synth-data-proj/data/downloads/3D_model_data.zip
100%|██████████| 42.5G/42.5G [32:49<00:00, 21.6MB/s]  


'data/downloads/3D_model_data.zip'

In [11]:
# Download the clipart data set
gdown.download(f"{BASE_URL}{CLIPART_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, 'clipart_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1f7yEK6WCaN_Af-opQ8c6Ey1JRQ-meMzE
From (redirected): https://drive.google.com/uc?id=1f7yEK6WCaN_Af-opQ8c6Ey1JRQ-meMzE&confirm=t&uuid=c189829e-ea4d-4e30-b870-be748b4b6d22&at=ABOpFIrgqN5Mqa6FNdAbyyfiyBbB%3A1761842595558
To: /home/developer/synth-data-proj/data/downloads/clipart_data.zip
100%|██████████| 42.3G/42.3G [17:16<00:00, 40.8MB/s]  


'data/downloads/clipart_data.zip'

In [12]:
# Download the genai data set
gdown.download(f"{BASE_URL}{GENAI_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, 'genai_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1yvGTLz4i4hapHEaW4cM8cvc1eTnTUMXc
From (redirected): https://drive.google.com/uc?id=1yvGTLz4i4hapHEaW4cM8cvc1eTnTUMXc&confirm=t&uuid=b02e8563-4ec5-4cef-83b6-e9c54c736bdf&at=ABOpFIpxiiD9Q1iv768KSCh4qtsQ%3A1761843647550
To: /home/developer/synth-data-proj/data/downloads/genai_data.zip
100%|██████████| 13.5G/13.5G [05:51<00:00, 38.3MB/s]


'data/downloads/genai_data.zip'

Unzip the files into the processed data directory.

In [13]:
# Get a list of the downloaded zip files
zip_files = [f for f in os.listdir(DOWNLOAD_PATH) if f.endswith('.zip')]
print(zip_files)

['baseline_data.zip', 'genai_data.zip', 'test_data.zip', '3D_model_data.zip', 'clipart_data.zip']


In [14]:
# For each zip file, unzip it into the processed data directory
for zip_file in zip_files:
    zip_file = os.path.join(DOWNLOAD_PATH, zip_file)
    # Unzip the file into the processed data directory
    with ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall("data/processed/")

Delete the zip files after their contents have been extracted.

## 3. Stage Data

Data is staged into a staging folder for the training process so modular chunks of data can be moved in and out as needed.

In [15]:
# Create the staging directory
os.makedirs(os.path.join('data', 'staging'), exist_ok=True)
os.makedirs(os.path.join('data', 'staging', 'train'), exist_ok=True)
os.makedirs(os.path.join('data', 'staging', 'val'), exist_ok=True)

In [16]:
# Get the list of training data csv files
trn_csv_files = {
    "3d_model": "data/processed/3d_models/train_data_splits.csv",
    "baseline": "data/processed/baseline/train_data_splits.csv",
    "clipart": "data/processed/clipart/train_data_splits.csv",
    "genai": "data/processed/genai/train_data_splits.csv"
}

In [17]:
# Read the files into a dataframe
t_model_3d_splits = pd.read_csv(trn_csv_files['3d_model'])
t_baseline_splits = pd.read_csv(trn_csv_files['baseline'])
t_clipart_splits = pd.read_csv(trn_csv_files['clipart'])
t_genai_splits = pd.read_csv(trn_csv_files['genai'])

In [18]:
# Get the list of validation data csv files
val_csv_files = {
    "3d_model": "data/processed/3d_models/val_data_splits.csv",
    "baseline": "data/processed/baseline/val_data_splits.csv",
    "clipart": "data/processed/clipart/val_data_splits.csv",
    "genai": "data/processed/genai/val_data_splits.csv"
}

In [19]:
# Read the files into a dataframe
v_model_3d_splits = pd.read_csv(val_csv_files['3d_model'])
v_baseline_splits = pd.read_csv(val_csv_files['baseline'])
v_clipart_splits = pd.read_csv(val_csv_files['clipart'])
v_genai_splits = pd.read_csv(val_csv_files['genai'])

In [20]:
def gen_data_mix(baseline_pct, modeled_pct, clipart_pct, genai_pct, stage='train', seed=42):
    """Generate a mixed dataset based on the specified percentages."""
    if stage == 'train':
        baseline_data = t_baseline_splits.sample(frac=baseline_pct, random_state=seed*10)
        model_3d_data = t_model_3d_splits.sample(frac=modeled_pct, random_state=seed*100)
        clipart_data = t_clipart_splits.sample(frac=clipart_pct, random_state=seed*1000)
        genai_data = t_genai_splits.sample(frac=genai_pct, random_state=seed*10000)
    else:
        baseline_data = v_baseline_splits.sample(frac=baseline_pct, random_state=seed*13)
        model_3d_data = v_model_3d_splits.sample(frac=modeled_pct, random_state=seed*133)
        clipart_data = v_clipart_splits.sample(frac=clipart_pct, random_state=seed*1333)
        genai_data = v_genai_splits.sample(frac=genai_pct, random_state=seed*13333)

    mixed_data = pd.concat([baseline_data, model_3d_data, clipart_data, genai_data]).reset_index(drop=True)
    return mixed_data

In [21]:
# For each file in the data series, copy it to the staging folder
def copy_to_staging(df, stage='train'):
    """Copy files listed in the dataframe to the staging directory."""
    for idx, row in df.iterrows():
        src_image_path = row['img_path']
        src_label_path = row['ann_path']
        
        # Convert paths from windows to unix if necessary
        src_image_path = src_image_path.replace('\\', '/')
        src_label_path = src_label_path.replace('\\', '/')

        # Define destination paths
        dest_image_path = os.path.join('data', 'staging', stage, os.path.basename(src_image_path))
        dest_label_path = os.path.join('data', 'staging', stage, os.path.basename(src_label_path))

        # Copy image and label files to staging directory
        shutil.copy(src_image_path, dest_image_path)
        shutil.copy(src_label_path, dest_label_path)

In [23]:
# Cleanup - delete all files in the staging folders
def cleanup_staging():
    """Delete all files in the staging directories."""
    # Delete .cache files in the staging folder
    for filename in os.listdir('data/staging'):
        if filename.endswith('.cache'):
            file_path = os.path.join('data', 'staging', filename)
            try:
                os.unlink(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')
                
    staging_dirs = [os.path.join('data', 'staging', 'train'), os.path.join('data', 'staging', 'val')]
    # Delete all files in the staging subdirectories
    for staging_dir in staging_dirs:
        for filename in os.listdir(staging_dir):
            file_path = os.path.join(staging_dir, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')

In [24]:
# Create the yaml file for YOLO model training
# Create filename and file path
yaml_filename = 'detect.yaml'
yaml_path = os.path.join('data', 'staging', yaml_filename)

# YAML content
yaml_content = """
path: data/staging  # dataset root dir (leave empty for HUB)
train: train  # train images (relative to 'path')
val:   val    # val images (relative to 'path')

names:
  0: drone
"""

# Write the YAML content to the file
with open(yaml_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

## 4. Train Model

**Experimental Design**

| **Run** | **Authentic Data** | **3D Model Data** | **Clipart Data** | **Gen AI Data** |
| --- | --- | --- | --- | --- |
| 1   | 26  | 27  | 23  | 24  |
| 2   | 0   | 57  | 22  | 21  |
| 3   | 0   | 27  | 73  | 0   |
| 4   | 27  | 0   | 0   | 73  |
| 5   | 27  | 33  | 40  | 0   |
| 6   | 38  | 0   | 33  | 29  |
| 7   | 38  | 33  | 29  | 0   |
| 8   | 0   | 0   | 76  | 24  |
| 9   | 0   | 24  | 52  | 24  |
| 10  | 100 | 0   | 0   | 0   |
| 11  | 0   | 100 | 0   | 0   |
| 12  | 27  | 73  | 0   | 0   |
| 13  | 0   | 0   | 30  | 70  |
| 14  | 29  | 0   | 36  | 35  |
| 15  | 70  | 30  | 0   | 0   |
| 16  | 72  | 0   | 0   | 28  |
| 17  | 0   | 0   | 100 | 0   |
| 18  | 26  | 35  | 0   | 39  |
| 19  | 0   | 27  | 0   | 73  |
| 20  | 0   | 70  | 30  | 0   |
| 21  | 40  | 30  | 0   | 30  |
| 22  | 27  | 0   | 73  | 0   |
| 23  | 70  | 0   | 30  | 0   |
| 24  | 0   | 0   | 0   | 100 |
| 25  | 0   | 26  | 27  | 47  |
| 26  | 0   | 72  | 0   | 28  |

In [ ]:
# List of the data combinations
doe_combinations = [
    (1.00, 0.00, 0.00, 0.00),
    (0.00, 1.00, 0.00, 0.00),
    (0.00, 0.00, 1.00, 0.00),
    (0.00, 0.00, 0.00, 1.00),
    
    (0.00, 0.57, 0.22, 0.21),
    (0.00, 0.27, 0.73, 0.00),
    (0.00, 0.00, 0.76, 0.24),
    (0.00, 0.24, 0.52, 0.24),
    (0.00, 0.00, 0.30, 0.70),
    (0.00, 0.27, 0.00, 0.73),
    (0.00, 0.70, 0.30, 0.00),
    (0.00, 0.26, 0.27, 0.47),
    (0.00, 0.72, 0.00, 0.28),
    
    (0.27, 0.00, 0.00, 0.73),
    (0.38, 0.00, 0.33, 0.29),
    (0.29, 0.00, 0.36, 0.35),
    (0.72, 0.00, 0.00, 0.28),
    (0.27, 0.00, 0.73, 0.00),
    (0.70, 0.00, 0.30, 0.00),
    
    (0.27, 0.73, 0.00, 0.00),
    (0.70, 0.30, 0.00, 0.00),
    (0.26, 0.35, 0.00, 0.39),
    (0.40, 0.30, 0.00, 0.30),
    
    (0.27, 0.33, 0.40, 0.00),
    (0.38, 0.33, 0.29, 0.00),
    
    (0.26, 0.27, 0.23, 0.24)
]

In [32]:
# Pick a random set from the DOE combinations
rng = np.random.default_rng()
doe_rng_choice = rng.choice(doe_combinations, size=7, replace=False)
print(doe_rng_choice)

[[       0.29           0        0.36        0.35]
 [       0.26        0.35           0        0.39]
 [       0.27        0.73           0           0]
 [        0.7         0.3           0           0]
 [          0           0        0.76        0.24]
 [        0.4         0.3           0         0.3]
 [       0.38        0.33        0.29           0]]


In [33]:
# Partition the DOE runs into small partitions for each node
doe_partition = [
    (0.29, 0.00, 0.36, 0.35),
    (0.26, 0.35, 0.00, 0.39),
    (0.70, 0.30, 0.00, 0.00),
    (0.27, 0.73, 0.00, 0.00),
    (0.00, 0.00, 0.76, 0.24),
    (0.40, 0.30, 0.00, 0.30),
    (0.38, 0.33, 0.29, 0.00)
]

In [34]:
# YOLO model config
config = {
    "lr0": 0.00099,
    "lrf": 0.01081,
    "momentum": 0.85521,
    "weight_decay": 0.00052,
    "hsv_h": 0.01634,
    "hsv_s": 0.7638,
    "hsv_v": 0.38363,
    "translate": 0.07243,
    "fliplr": 0.5,
    "mosaic": 0.95755
}

In [ ]:
# Loop through the combinations
for idx, (baseline_pct, modeled_pct, clipart_pct, genai_pct) in enumerate(doe_partition):
    print(f"Processing combination {idx+1}: Baseline={baseline_pct}, 3D Model={modeled_pct}, Clipart={clipart_pct}, GenAI={genai_pct}")
    
    # Create a suffix for the file names
    suffix = f"{int(baseline_pct*100)}-{int(modeled_pct*100)}-{int(clipart_pct*100)}-{int(genai_pct*100)}"
    
    # Cleanup staging directory
    cleanup_staging()
    
    # Generate mixed training data
    mixed_train_data = gen_data_mix(baseline_pct, modeled_pct, clipart_pct, genai_pct, stage='train', seed=42*(idx+1))
    copy_to_staging(mixed_train_data, stage='train')
    mixed_train_data.to_csv(os.path.join('data', 'staging', f'train_data_{suffix}.csv'), index=False)

    # Generate mixed validation data
    mixed_val_data = gen_data_mix(baseline_pct, modeled_pct, clipart_pct, genai_pct, stage='val', seed=84*(idx+1))
    copy_to_staging(mixed_val_data, stage='val')
    mixed_val_data.to_csv(os.path.join('data', 'staging', f'val_data_{suffix}.csv'), index=False)

    # Load a YOLOv11 model
    model = YOLO("yolo11x.pt")  # Load a pre-trained YOLOv11 model

    # Train the model
    results = model.train(
        data='data/staging/detect.yaml',
        batch=0.85,
        epochs=200,
        patience=20,
        optimizer="AdamW",
        device=0,
        name=f"model_{suffix}",
        **config
    )

Processing combination 1: Baseline=0.5, 3D Model=0.0, Clipart=0.5, GenAI=0.0
New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.222 🚀 Python-3.11.0rc1 torch-2.9.0+cu128 CUDA:0 (NVIDIA H100 NVL, 95330MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.85, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/staging/detect.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01634, hsv_s=0.7638, hsv_v=0.38363, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00099, lrf=0.01081, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yol

KeyboardInterrupt: 